In [2]:
import numpy as np
import pandas
import time
import elpigraph
#import elpigraphgpu # If you have a GPU. Requires Cupy
import matplotlib.pyplot as plt
from copy import deepcopy
import rpy2.robjects.packages as rpackages
import rpy2.robjects
import rpy2.robjects.numpy2ri
import rpy2.robjects.pandas2ri
r_elpigraph = rpackages.importr("ElPiGraph.R")
rpy2.robjects.numpy2ri.activate()
rpy2.robjects.pandas2ri.activate()
plt.style.use('seaborn')
np.random.seed(0)

# I - Speed comparison

In [ ]:
pandas.read_csv('/home/utilisateur/', sep='\t')

In [4]:
### Python
np.random.seed(0)
num_points = [1000,10000,100000]
num_nodes = [10,20,30,40,50,60]

run_points = []
for j in num_points:
    run_nodes = []
    for i in num_nodes:
        X=np.random.random(size=(j,10))
        s = time.time()
        res = elpigraph.computeElasticPrincipalTree(X = X,NumNodes = i,drawPCAView=False)
        end = time.time() - s
        run_nodes.append(end)
    run_points.append(run_nodes)

Generating the initial configuration
Creating a chain in the 1st PC with 2 nodes
90% of the points have been used as initial conditions. Resetting.
Constructing tree 1 of 1 / Subset 1 of 1
Performing PCA
Using standard PCA
10 dimensions are being used
100.0 % of the original variance has been retained
The elastic matrix is being used. Edge configuration will be ignored
Computing EPG with  10  nodes on  1000  points and  10  dimensions
Nodes =  2 3 4 5 6 7 8 9 

BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NSTARS	NRAYS	NRAYS2	MSE	MSEP	FVE	FVEP	UE	UR	URN	URN2	URSD

1|0|0|0|0|0|0||10	0.6063	10	9	0	0	0	0	0.5868	0.5821	0.3003	0.306	0.0194	0.0	0.0004	0.0036	0


MSDEnergyPlot not yet implemented
accuracyComplexityPlot not yet implemented
0.4939  seconds elapsed
Generating the initial configuration
Creating a chain in the 1st PC with 2 nodes
90% of the points have been used as initial conditions. Resetting.
Constructing tree 1 of 1 / Subset 1 of 1
Performing PCA
Using standard PCA
10 dimensions are bein

In [ ]:
### Python hybrid
# np.random.seed(0)
# num_points = [1000,10000,100000]
# num_nodes = [10,20,30,40,50,60]

# run_points = []
# for j in num_points:
#     run_nodes = []
#     for i in num_nodes:
#         X=np.random.random(size=(j,10))
#         s = time.time()
#         res = elpigraphgpu.computeElasticPrincipalTree(X = X,NumNodes = i,drawPCAView=False)
#         end = time.time() - s
#         run_nodes.append(end)
#     run_points.append(run_nodes)

In [5]:
### R
np.random.seed(0)
num_points = [1000,10000,100000]
num_nodes = [10,20,30,40,50,60]

run_points_r = []
for j in num_points:
    run_nodes = []
    for i in num_nodes:
        X=np.random.random(size=(j,10))
        s = time.time()
        res= r_elpigraph.computeElasticPrincipalTree(X = X,NumNodes = i)
        end = time.time() - s
        run_nodes.append(end)
    run_points_r.append(run_nodes)

[1] "Generating the initial configuration"
[1] "Creating a chain in the 1st PC with 2 nodes"
[1] "Constructing tree 1 of 1 / Subset 1 of 1"
[1] "Performing PCA on the data"
[1] "Using standard PCA"
[1] "10 dimensions are being used"
[1] "100% of the original variance has been retained"
[1] "The elastic matrix is being used. Edge configuration will be ignored"
[1] "Computing EPG with 10 nodes on 1000 points and 10 dimensions"
[1] "Using a single core"
Nodes = 2 3 4 5 6 7 8 9 
BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NSTARS	NRAYS	NRAYS2	MSE	MSEP	FVE	FVEP	UE	UR	URN	URN2	URSD
1|0|0|0|0|0|0||10	0.6063	10	9	0	0	0	0	0.5868	0.5821	0.3003	0.306	0.01943	3.598e-05	0.0003598	0.003598	0
1.678 sec elapsed
[[1]]

[1] "Generating the initial configuration"
[1] "Creating a chain in the 1st PC with 2 nodes"
[1] "Constructing tree 1 of 1 / Subset 1 of 1"
[1] "Performing PCA on the data"
[1] "Using standard PCA"
[1] "10 dimensions are being used"
[1] "100% of the original variance has been retained"
[1] "The el

In [6]:
### Plotting
for i in range(len(num_points)):

    #plt.plot(num_nodes,np.array(run_points_colab_hybrid[i])/60,marker='.') # run hybrid version if you have a gpu (or get results from colab)
    plt.plot(num_nodes,np.array(run_points[i])/60,marker='.')
    plt.plot(num_nodes,np.array(run_points_r[i])/60,marker='.')

    plt.xlabel('Number of nodes',fontsize=16)
    plt.ylabel('Time (minutes)',fontsize=16)
    plt.legend(['Python_Hybrid_cpu_gpu','Python_one_cpu','R_one_cpu'],fontsize=13)
    plt.title('Number of points (10 dimensions) : '+str(num_points[i]),fontsize=16)
    plt.show()

NameError: name 'num_points' is not defined

# II - Checking output
### Step 1 :  generate output for R and Python

In [12]:
# Load example data
X =  np.genfromtxt('data/tree_data.csv', delimiter=',')

# Create desired list of inputs for R and Python
input_data = [X]*5
epg_n_nodes = [20,25,15,10,20]
epg_lambda = [.1,.02,.7,.03,.08]
epg_mu = [.02,.07,.01,.04,.06]
epg_trimmingradius = [float('inf'),.7,.8,.6,.5]
epg_finalenergy = ['Penalized','Base','Penalized','Base','Base']
epg_alpha = [.01,.03,.05,.08,.04]
epg_beta = [.03,.02,.04,.07,.01]
epg_mode = [2,1,1,2,1]
epg_n_processes = [1,2,1,2,1]
epg_collapse_mode = ['PointNumber','PointNumber_Extrema','PointNumber_Leaves','EdgesNumber','EdgesLength']
epg_collapse_par = [5,7,4,6,3]
epg_maxsteps = [float('inf'),1000,100,20,200]
                                  # Python uses WeightedCentroid not Weigthed (corrected typo)
epg_ext_mode =   ['QuantDists','QuantCentroid','WeightedCentroid','QuantCentroid','WeightedCentroid']
r_epg_ext_mode = ['QuantDists','QuantCentroid','WeigthedCentroid','QuantCentroid','WeigthedCentroid'] 
epg_ext_par = [.5,.6,.8,.9,.5]
epg_shift_mode = ['NodeDensity','NodePoints','NodeDensity','NodePoints','NodeDensity']
epg_shift_radius = [0.05,0.07,0.04,0.08,0.03]
epg_shift_max = [5,7,4,8,6]



# Results storage Python
epg_main = []
epg_obj_collapse = []
epg_obj_shift = []
epg_obj_extend = []
epg_obj_fineTune = []
# Results storage R
r_epg_main = []
r_epg_obj_collapse = []
r_epg_obj_shift = []
r_epg_obj_extend = []
r_epg_obj_fineTune = []

for i in range(len(input_data)):
    
    ############################ Run functions, Python version ###################################
    
    epg_main.append(elpigraph.computeElasticPrincipalTree(X=input_data[i],NumNodes = epg_n_nodes[i], 
                                                          Lambda=epg_lambda[i], Mu=epg_mu[i],
                                                          TrimmingRadius = epg_trimmingradius[i],
                                                          FinalEnergy = epg_finalenergy[i],
                                                          alpha = epg_alpha[i],
                                                          beta = epg_beta[i],                                                    
                                                          Do_PCA=False,CenterData=False,
                                                          n_cores = epg_n_processes[i],
                                                          nReps=1,
                                                          EmbPointProb=1.0,
                                                          drawPCAView=False,
                                                          Mode = epg_mode[i],
                                                          MaxSteps = epg_maxsteps[i])[0])
    
    # util functions input
    epg_obj = epg_main[i]
    init_nodes_pos = epg_obj['NodePositions']
    init_edges = epg_obj['Edges'][0]
    #########################################
    try:
        epg_obj_collapse.append(elpigraph.CollapseBranches(X = input_data[i], PG = epg_obj, Mode = epg_collapse_mode[i], ControlPar = epg_collapse_par[i]))
    except:
        epg_obj_collapse.append('bug')

    epg_obj_shift.append(elpigraph.ShiftBranching(X = input_data[i], 
                                                  PG = epg_obj, 
                                                  TrimmingRadius = epg_trimmingradius[i],                       
                                                  SelectionMode = epg_shift_mode[i], 
                                                  DensityRadius = epg_shift_radius[i],
                                                  MaxShift = epg_shift_max[i]))
    
    epg_obj_extend.append(elpigraph.ExtendLeaves(X = input_data[i], 
                                                 PG = epg_obj,
                                                 TrimmingRadius = epg_trimmingradius[i],
                                                 Mode = epg_ext_mode[i], 
                                                 ControlPar = epg_ext_par[i],
                                                 PlotSelected = False,
                                                 DoSA_maxiter=4000)) #number of iterations for simulated annealing
    
    epg_obj_fineTune.append(elpigraph.fineTuneBR(X=input_data[i],
                                                MaxSteps = epg_maxsteps[i],
                                                Mode = 2,
                                                NumNodes = epg_n_nodes[i], 
                                                InitNodePositions = init_nodes_pos,
                                                InitEdges=init_edges,
                                                Lambda=epg_lambda[i], Mu=epg_mu[i],
                                                TrimmingRadius= epg_trimmingradius[i],
                                                FinalEnergy = epg_finalenergy[i],
                                                alpha = epg_alpha[i],
                                                beta = epg_beta[i],                                                    
                                                Do_PCA=False,CenterData=False,
                                                drawAccuracyComplexity = False, drawEnergy = False,drawPCAView = False,
                                                n_cores = epg_n_processes[i],
                                                nReps=1,
                                                ProbPoint=1.0)[0])
    
    ############################ Run functions, R version ###################################

    tmp = r_elpigraph.computeElasticPrincipalTree(X=input_data[i],NumNodes = epg_n_nodes[i], 
                                                  Lambda=epg_lambda[i], Mu=epg_mu[i],
                                                  TrimmingRadius= epg_trimmingradius[i],
                                                  FinalEnergy = epg_finalenergy[i],
                                                  alpha = epg_alpha[i],
                                                  beta = epg_beta[i],                                                    
                                                  Do_PCA=False,CenterData=False,
                                                  n_cores = epg_n_processes[i],
                                                  nReps=1,
                                                  ProbPoint=1.0,
                                                  drawPCAView=False,
                                                  Mode = epg_mode[i],
                                                  MaxSteps = epg_maxsteps[i])[0]
    
    r_epg_main.append(dict(zip(tmp.names, map(list,np.array(tmp))))) # Convert R result to dict format
    
    # util functions input
    r_epg_obj = tmp
    init_nodes_pos = r_epg_obj[0]
    init_edges = r_epg_obj[1][0]
    #########################################
#     obj_collapse = deepcopy(r_epg_obj)
    try:
        r_epg_obj_collapse.append(r_elpigraph.CollapseBrances(X = input_data[i], TargetPG = r_epg_obj, Mode = epg_collapse_mode[i], ControlPar = epg_collapse_par[i]))
    except:
        r_epg_obj_collapse.append('bug')

    r_epg_obj_shift.append(r_elpigraph.ShiftBranching(X = input_data[i], 
                                                      TargetPG = r_epg_obj, 
                                                      TrimmingRadius = epg_trimmingradius[i],                       
                                                      SelectionMode = epg_shift_mode[i], 
                                                      DensityRadius = epg_shift_radius[i],
                                                      MaxShift = epg_shift_max[i]))
    
#     obj_extend = deepcopy(r_epg_obj)
    tmp_ext = r_elpigraph.ExtendLeaves(X = input_data[i], 
                                       TargetPG = r_epg_obj,
                                       TrimmingRadius = epg_trimmingradius[i],
                                       Mode = r_epg_ext_mode[i], 
                                       ControlPar = epg_ext_par[i],
                                       PlotSelected = False)
    r_epg_obj_extend.append(dict(zip(tmp_ext.names, map(list,np.array(tmp_ext)))))
    
    tmp_fineTune = r_elpigraph.fineTuneBR(X=input_data[i],
                                          MaxSteps = epg_maxsteps[i],
                                          Mode = 2,
                                          NumNodes = epg_n_nodes[i], 
                                          InitNodePositions = init_nodes_pos,
                                          InitEdges=init_edges,
                                          Lambda=epg_lambda[i], Mu=epg_mu[i],
                                          TrimmingRadius= epg_trimmingradius[i],
                                          FinalEnergy = epg_finalenergy[i],
                                          alpha = epg_alpha[i],
                                          beta = epg_beta[i],                                                    
                                          Do_PCA=False,CenterData=False,
                                          drawAccuracyComplexity = False, drawEnergy = False,drawPCAView = False,
                                          n_cores = epg_n_processes[i],
                                          nReps=1,
                                          ProbPoint=1.0)[0]
    r_epg_obj_fineTune.append(dict(zip(tmp_fineTune.names, map(list,np.array(tmp_fineTune)))))

Generating the initial configuration
Creating a chain in the 1st PC with 2 nodes
90% of the points have been used as initial conditions. Resetting.
Constructing tree 1 of 1 / Subset 1 of 1
The elastic matrix is being used. Edge configuration will be ignored
Computing EPG with  20  nodes on  492  points and  3  dimensions
Nodes =  2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NSTARS	NRAYS	NRAYS2	MSE	MSEP	FVE	FVEP	UE	UR	URN	URN2	URSD

2||20	0.0841	20	19	14	2	0	0	0.0357	0.0343	0.9337	0.9363	0.0475	0.0008	0.0165	0.3296	0


MSDEnergyPlot not yet implemented
accuracyComplexityPlot not yet implemented
0.676  seconds elapsed
Removing the terminal branch with nodes: [14 17]
Moving the branching point at node 0
Moving the branching point at node 14
Performing simulated annealing. This may take a while
Performing simulated annealing. This may take a while
Performing simulated annealing. This may take a while
Performing simulated annealing. This may take a whil

R[write to console]: Error in NodeMat[unlist(igraph::V(Ret_Net)$name), ] : 
  no 'dimnames' attribute for array
Calls: <Anonymous>

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In ElPiGraph.R:::ElPrincGraph(X = X, NumNodes = NumNodes, NumEdges = NumEdges,  :
R[write to console]: 
 
R[write to console]:  When setting GrammarOptimization to TRUE, MaxSteps must be finite. Using MaxSteps = 1



[1] "Moving the branching point at node 2"
[1] "6 points selected to compute the centroid while extending node 7"
[1] "4 points selected to compute the centroid while extending node 8"
[1] "3 points selected to compute the centroid while extending node 10"
[1] "Creating a sock cluster with 2 nodes"
[1] "Constructing tree 1 of 1 / Subset 1 of 1"
[1] "Computing EPG with 10 nodes on 492 points and 3 dimensions"
[1] "Using grammar optimization"
[1] "Using a user supplied cluster. It must contains the data points in a matrix X"
[1] "Exporting the additional variables to the cluster"
Nodes = 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 
BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NSTARS	NRAYS	NRAYS2	MSE	MSEP	FVE	FVEP	UE	UR	URN	URN2	URSD
1||10	0.08113	10	9	6	1	0	0	0.05067	Inf	0.906	-Inf	0.02377	0.006688	0.06688	0.6688	0
0.429 sec elapsed
Generating the initial configuration
Creating a chain in the 1st PC with 2 nodes
90% of the points have been used as initial conditions. Resetting.


### Step 2 : check final output : computePrincipalTree, ExtendLeaves, fineTuneBR
#### For dict keys in (NodePositions, Edges, FinalReport, ElasticMatrix) -> prints key, iteration index, function if a difference is found in the result dictionary

In [13]:
funcs = ['computeElasticPrincipalTree','ExtendLeaves','fineTuneBR']
j = 0  #funcs index
for res_py,res_R in [(epg_main,r_epg_main),(epg_obj_extend,r_epg_obj_extend),(epg_obj_fineTune,r_epg_obj_fineTune)]: # for func in funcs
    for i in range(len(input_data)): # check each set of output
        one_res_py = res_py[i]
        one_res_R = res_R[i]
        
        for key in one_res_py:
            if key == 'NodePositions':
                try: assert np.allclose(one_res_py[key], one_res_R[key])

                except: print(key,i,funcs[j])

            if key == 'Edges':
                try: assert all(map(lambda x:np.all(x),[one_res_py[key][0]==(one_res_R[key][0]-1), #correcting R indexing that starts at one
                                                        one_res_py[key][1][~np.isnan(one_res_py[key][1])]==one_res_R[key][1][~np.isnan(one_res_R[key][1])],
                                                        one_res_py[key][2][~np.isnan(one_res_py[key][2])]==one_res_R[key][2][~np.isnan(one_res_R[key][2])]]))

                except: print(key,i,funcs[j])

            if key == 'FinalReport':
                try: assert(np.allclose(np.array(list(one_res_py[key].values()))[1:].astype(float), 
                                        np.array(one_res_R[key]).flatten()[1:].astype(float)))
                except: print(key,i, funcs[j])

            if key == 'ElasticMatrix':
                try: assert np.all(one_res_py[key] == one_res_R[key])
                except: print(key,i,funcs[j])
    j+=1

NodePositions 0 ExtendLeaves


### Step 3 : check CollapseBranches, ShiftBranching

In [78]:
for i in range(len(input_data)):
    
    # CollapseBranches         
    try: 
        r_collapse_dict = dict(zip(r_epg_obj_collapse[i].names,r_epg_obj_collapse[i])) 
        assert all([np.all(np.array(epg_obj_collapse[i]['Edges'])==(r_collapse_dict['Edges']-1)), #correcting R indexing that starts at one
                     np.allclose(epg_obj_collapse[i]['Nodes'],r_collapse_dict['Nodes'])])
    except: 
        #Collapse branches can bug for input parameters, or return empty array. We check one of these cases happened for both versions
        try: 
            for obj in [epg_obj_collapse[i],r_epg_obj_collapse[i]]:
                if type(obj) ==str:
                    assert obj == 'bug'  #check if function bugged
                elif type(obj) ==  dict: 
                    assert np.array(obj['Edges']).size == 0    #or check if array is empty, Python
                elif hasattr(np.array(obj[0]),'size'):
                    assert np.array(obj[0]).size == 0          #or check if array is empty, R
            print('CollapseBranches failed for both versions',i)
        except: print('CollapseBranches', i)
        

        
    # ShiftBranching
    r_shift_dict = dict(zip(r_epg_obj_shift[i].names,r_epg_obj_shift[i]))          
    try: assert all([np.all(np.array(epg_obj_shift[i]['Edges'])==(r_shift_dict['Edges']-1)),       #correcting R indexing that starts at one
                     np.allclose(epg_obj_shift[i]['NodePositions'],r_shift_dict['NodePositions'])])
    except: 
        # Edges might be correct but differ in ordering. We double-check that :
        sort_shift_py = epg_obj_shift[i]['Edges'][:,1].argsort()
        sort_shift_r = (r_shift_dict['Edges']-1)[:,1].argsort()
        try: assert all([np.all(np.array(epg_obj_shift[i]['Edges'][sort_shift_py])==(r_shift_dict['Edges']-1)[sort_shift_r]),  #correcting R indexing that starts at one
                         np.allclose(epg_obj_shift[i]['NodePositions'],r_shift_dict['NodePositions'])])
        except: print('ShiftBranching',i)

CollapseBranches 3
CollapseBranches failed for both versions 4


In [80]:
obj

{'Edges': [], 'Nodes': array([], shape=(0, 3), dtype=float64)}

In [83]:
i = 3
for obj in [epg_obj_collapse[i],r_epg_obj_collapse[i]]:
    if type(obj) ==str:
        assert obj == 'bug'  #check if function bugged
    elif type(obj) ==  dict: 
        assert np.array(obj['Edges']).size == 0    #or check if array is empty, Python
    elif hasattr(np.array(obj[0]),'size'):
        assert np.array(obj[0]).size == 0  #or check if array is empty, R
print('CollapseBranches failed for both versions',i)

CollapseBranches failed for both versions 3


In [82]:
obj[0]

'b'

In [47]:
dict(zip(r_epg_obj_collapse[i].names,r_epg_obj_collapse[i]))

{'Edges': array([], shape=(0, 2), dtype=float64),
 'Nodes': array([], shape=(0, 3), dtype=float64)}

In [41]:
obj

Edges,[RTYPES.REALSXP]
Nodes,[RTYPES.REALSXP]


In [33]:
i = 3
((epg_obj_collapse[i] == 'bug' or epg_obj_collapse[i]['Edges'].size == 0) and
                  (r_epg_obj_collapse[i] == 'bug' or r_epg_obj_collapse[i]['Edges'].size == 0)) 

AttributeError: 'list' object has no attribute 'size'

In [36]:
r_epg_obj_collapse[3]

'bug'

In [29]:
r_epg_obj_collapse[4][1] == np.array([])

/home/utilisateur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [14]:
i=0
r_collapse_dict = dict(zip(r_epg_obj_collapse[i].names,r_epg_obj_collapse[i]))          
np.array(epg_obj_collapse[i]['Edges'])

array([[ 0,  8],
       [ 0, 12],
       [ 0, 18],
       [ 1,  8],
       [ 1, 16],
       [ 2,  4],
       [ 2, 12],
       [ 3,  5],
       [ 3, 10],
       [ 4,  7],
       [ 5,  6],
       [ 6, 14],
       [ 7,  9],
       [ 9, 11],
       [10, 18],
       [11, 13],
       [14, 15],
       [16, 17]])

In [64]:
import copy

In [15]:
np.allclose(np.array(r_epg_main[0]['NodePositions']),epg_main[0]['NodePositions'])

True

In [72]:
X
PG = epg_main[0]
Mode = "PointNumber"
ControlPar = 5
TrimmingRadius = float('inf')

TargetPG=copy.deepcopy(PG)

# Generate net
Net = elpigraph.src.graphs.ConstructGraph(PrintGraph = TargetPG)

# Set a color for the edges
Net.es.set_attribute_values('status','keep')

# Get the leaves
Leaves = np.where(np.array(Net.degree(mode = "all"))==1)[0]

# get the partition
PartStruct = elpigraph.src.core.PartitionData(X = X, NodePositions = TargetPG['NodePositions'],MaxBlockSize=100000000, TrimmingRadius = TrimmingRadius,SquaredX=np.sum(X**2,axis=1,keepdims=1))

# Project points onto the graph
ProjStruct = elpigraph.src.reporting.project_point_onto_graph(X = X,
                                     NodePositions = TargetPG['NodePositions'],
                                     Edges = TargetPG['Edges'][0],
                                     Partition = PartStruct[0])

# get branches
Branches = elpigraph.src.graphs.GetSubGraph(Net = Net, Structure = 'branches')

# get the number of points on the different branches
AllBrInfo = []
for BrNodes in Branches:
    # print(BrNodes)
    PotentialPoints = np.array([False] * len(ProjStruct['EdgeID']))
    NodeNames = np.array(BrNodes)

    # Get the points on the extrema
    StartEdg = np.where(np.any(ProjStruct['Edges'] == NodeNames[0],axis=1))[0]

    StartOnNode = np.array([False] * len(ProjStruct['EdgeID']))

    SelPoints = np.isin(ProjStruct['EdgeID'], StartEdg)
    StartOnNode[SelPoints] = (ProjStruct['ProjectionValues'][SelPoints] > 1) | (ProjStruct['ProjectionValues'][SelPoints] < 0)

    EndEdg = np.where(np.any(ProjStruct['Edges'] == NodeNames[-1],axis=1))[0]
    EndOnNode = np.array([False] * len(ProjStruct['EdgeID']))

    SelPoints = np.isin(ProjStruct['EdgeID'], EndOnNode)
    EndOnNode[SelPoints] = (ProjStruct['ProjectionValues'][SelPoints] > 1) | (ProjStruct['ProjectionValues'][SelPoints] < 0)

    EdgLen = 0

    # Get the points on the branch (extrema are excluded)
    for i in range(1,len(BrNodes)):

        # Get the edge on the segment
        WorkingEdg = np.where(list(map(lambda x: all(np.isin(x , NodeNames[range((i-1),i+1)])),ProjStruct['Edges'])))[0].squeeze()
        # Get the len of the segment
        EdgLen = EdgLen + ProjStruct['EdgeLen'][WorkingEdg]

        # Get the points on the segment
        Points = ProjStruct['EdgeID'] == WorkingEdg
        Points[np.isnan(Points)] = False

        # Is the edge in the right direction?
        if(all(ProjStruct['Edges'][WorkingEdg, ] == NodeNames[range((i-1),i+1)])):
            Reverse = False
        else:
            Reverse = True

        # Counting points at the beginning

        if(i == 1 and len(BrNodes) > 2):
            if(Reverse):
                PotentialPoints[Points] = (ProjStruct['ProjectionValues'][Points] < 1) | PotentialPoints[Points]
            else :
                PotentialPoints[Points] = (ProjStruct['ProjectionValues'][Points] > 0) | PotentialPoints[Points]
            continue

        # Counting points at the end
        if(i == (len(BrNodes)-1)):
            if(Reverse):
                PotentialPoints[Points] = (ProjStruct['ProjectionValues'][Points] > 0) | PotentialPoints[Points]

            else :
                PotentialPoints[Points] = (ProjStruct['ProjectionValues'][Points] < 1) | PotentialPoints[Points]
            continue


        # all the other cases
        PotentialPoints[Points] = ((ProjStruct['ProjectionValues'][Points] > 0) & (ProjStruct['ProjectionValues'][Points] < 1)) | PotentialPoints[Points] 

    PointsOnEdgesLeaf = PotentialPoints

    if(np.isin(NodeNames[0], Leaves)):
        PointsOnEdgesLeaf = PointsOnEdgesLeaf | StartOnNode


    if(np.isin(NodeNames[-1],  Leaves)):
        PointsOnEdgesLeaf = PointsOnEdgesLeaf | EndOnNode

    AllBrInfo.append(dict(

            PointsOnEdges = sum(PotentialPoints),
            PointsOnEdgeExtBoth = sum(PotentialPoints | StartOnNode | EndOnNode),
            PointsOnEdgesLeaf = sum(PointsOnEdgesLeaf),
            EdgesCount = len(BrNodes) - 1,
            EdgesLen = EdgLen
        )
    )

# Now all the information has been pre-computed and it is possible to filter

if(Mode == "PointNumber"):
    ToFilter = np.array([i['PointsOnEdges'] for i in AllBrInfo]) < ControlPar


if(Mode == "PointNumber_Extrema"):
    ToFilter = np.array([i['PointsOnEdgeExtBoth'] for i in AllBrInfo]) < ControlPar


if(Mode == "PointNumber_Leaves"):
    ToFilter = np.array([i['PointsOnEdgesLeaf'] for i in AllBrInfo]) < ControlPar


if(Mode == "EdgesNumber"):
    ToFilter = np.array([i['EdgesCount'] for i in AllBrInfo]) < ControlPar


if(Mode == "EdgesLength"):
    ToFilter = np.array([i['EdgesLen'] for i in AllBrInfo]) < ControlPar


# Nothing to filter
if(sum(ToFilter)==0):
    print(
        dict(
            Edges = TargetPG['Edges'][0],
            Nodes = TargetPG['NodePositions']
        )
    )

# TargetPG_New = TargetPG
# NodesToRemove = None

# Keep track of all the nodes to remove
AllNodes_InternalBranches = {}
# For all the branches
for i in range(len(ToFilter)):

    # If we need to filter this
    if(ToFilter[i] == True):

        NodeNames = np.array(Branches[i])

        # Is it a final branch ? 
        if(any(np.isin(NodeNames[[0,-1]], Leaves))):
            # It's a terminal branch, we can safely take it out

            print("Removing the terminal branch with nodes:", NodeNames)

            if(len(NodeNames) > 2):
                NodeNames_Ext = [NodeNames[0]]
                NodeNames_Ext.extend(list(np.repeat(NodeNames[1:-1], 2)))
                NodeNames_Ext.append(NodeNames[-1])
            else :
                NodeNames_Ext = NodeNames
            # Set edges to be removed
            for e in range(0,len(NodeNames_Ext),2):
                rm_eid = Net.get_eid(NodeNames_Ext[e],NodeNames_Ext[e+1])
                Net.es[rm_eid]['status'] = "remove"

        else:
            # It's a "bridge". We cannot simply remove nodes. Need to introduce a new one by "fusing" two stars
            print("Removing the bridge branch with nodes:", NodeNames)

            # Update the list of nodes to update
            AllNodes_InternalBranches = list(set(AllNodes_InternalBranches).union(NodeNames))

# Create the network that will contain the final filtered network
Ret_Net = copy.deepcopy(Net)

# Get a net with all the groups of bridges to remove
tNet = Net.induced_subgraph(AllNodes_InternalBranches)

if(tNet.vcount()>0):
    # Get the different connected components
    CC = tNet.components()

    # Get the nodes associated with the connected components
    Member_Comps = Net.components().membership
    Vertex_Comps = [np.where(Member_Comps==i)[0] for i in np.unique(Member_Comps)]
    # Get the centroid of the different connected components
    Centroids = np.array([np.mean(TargetPG['NodePositions'][np.array(i)],axis=0) for i in Vertex_Comps])

    # Prepare a vector that will be used to contract vertices
    CVet = np.array(range(Net.vcount()))

    # For each centroid
    for i in range(len(Vertex_Comps)):
        # Add a new vertex
        Ret_Net = Ret_Net.add_vertex(Ret_Net.vcount())
        # Add a new element to the contraction vector
        CVet = np.append(CVet,len(CVet))
        #specify the nodes that will collapse on the new node
        CVet[Vertex_Comps[i]] = len(CVet)

    # collapse the network
    Ret_Net.contract_vertices(mapping = CVet)

# delete edges belonging to the terminal branches
edge_list = Ret_Net.get_edgelist()
Ret_Net.delete_edges([edge_list[i] for i in range(len(edge_list)) if Ret_Net.es['status'][i] =='remove'])
# remove loops that may have been introduced because of the collapse
Ret_Net.simplify(loops = True)
# Remove empty nodes
Ret_Net = Ret_Net.induced_subgraph(np.array(Ret_Net.vs.indices)[np.array(Ret_Net.degree())>0])

if(tNet.vcount()>0):
    NodeMat = np.vstack((TargetPG['NodePositions'], Centroids))

else :
    NodeMat = TargetPG['NodePositions']


NodeMat = NodeMat[Ret_Net.vs['name'],:]

# return(dict(Edges = np.array(Ret_Net.get_edgelist()),
#             Nodes = NodeMat))

Removing the terminal branch with nodes: [14 17]


In [137]:
Edges = Ret_Net.get_edgelist()
print(np.array(Edges)==EdgesR-1)
print(NodeMat==NodesR)

[[ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]]
[[False False  True]
 [False False False]
 [False False  True]
 [False False  True]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [ True False False]
 [False False False]
 [ True False  True]
 [False False  True]
 [False  True  True]
 [False  True False]
 [False False False]
 [False False False]
 [False False False]
 [False False False]]


In [136]:
%R -o EdgesR -o NodesR

In [24]:
NodePositions=np.array(r_epg_main[0]['NodePositions'])
Edges=r_epg_main[0]['Edges'][0]

In [141]:
i=0
r_collapse_dict = dict(zip(r_epg_obj_collapse[i].names,r_epg_obj_collapse[i]))          

In [144]:
r_collapse_dict['Edges']==EdgesR

/home/utilisateur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [25]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [26]:
%%R -i NodePositions -i Edges -i X -i resR

Mode = "PointNumber"
ControlPar = 5
TrimmingRadius = Inf

TargetPG = list(NodePositions=NodePositions,Edges=list(Edges=Edges))

# Generate net
Net <- ConstructGraph(PrintGraph = TargetPG)

# Set a color for the edges
igraph::E(Net)$status <- "keep"

# Get the leaves
Leaves <- names(which(igraph::degree(Net, mode = "all")==1))

# get the partition
PartStruct <- PartitionData(X = X, NodePositions = TargetPG$NodePositions, TrimmingRadius = TrimmingRadius)

# Project points ont the graph
ProjStruct <- project_point_onto_graph(X = X,
                                     NodePositions = TargetPG$NodePositions,
                                     Edges = TargetPG$Edges$Edges,
                                     Partition = PartStruct$Partition)

# get branches
Branches <- GetSubGraph(Net = Net, Structure = 'branches')

# get the number of points on the different branches
AllBrInfo <- lapply(Branches, function(BrNodes){

# print(BrNodes)

PotentialPoints <- rep(FALSE, length(ProjStruct$EdgeID))

NodeNames <- as.integer(names(BrNodes))

# Get the points on the extrema

StartEdg <- which(apply(ProjStruct$Edges == NodeNames[1], 1, any))
StartOnNode <- rep(FALSE, length(ProjStruct$EdgeID))

SelPoints <- ProjStruct$EdgeID %in% StartEdg
StartOnNode[SelPoints] <- ProjStruct$ProjectionValues[SelPoints] > 1 | ProjStruct$ProjectionValues[SelPoints] < 0

EndEdg <- which(apply(ProjStruct$Edges == NodeNames[length(NodeNames)], 1, any))
EndOnNode <- rep(FALSE, length(ProjStruct$EdgeID))

SelPoints <- ProjStruct$EdgeID %in% EndOnNode
EndOnNode[SelPoints] <- ProjStruct$ProjectionValues[SelPoints] > 1 | ProjStruct$ProjectionValues[SelPoints] < 0

EdgLen <- 0

# Get the points on the branch (extrema are excluded)
for(i in 2:length(BrNodes)){

  # Get the edge on the segment
  WorkingEdg <- which(apply(ProjStruct$Edges, 1, function(x){all(x %in% NodeNames[(i-1):i])}))

  # Get the length of the segment
  EdgLen <- EdgLen + ProjStruct$EdgeLen[WorkingEdg]

  # Get the points on the segment
  Points <- ProjStruct$EdgeID == WorkingEdg
  Points[is.na(Points)] <- FALSE

  # Is the edge in the right direction?
  if(all(ProjStruct$Edges[WorkingEdg, ] == NodeNames[(i-1):i])){
    Reverse <- FALSE
  } else {
    Reverse <- FALSE
  }

  # Counting points at the begining
  if(i == 2 & length(BrNodes) > 2){
    if(Reverse){
      PotentialPoints[Points] <- (ProjStruct$ProjectionValues[Points] < 1) | PotentialPoints[Points]
    } else {
      PotentialPoints[Points] <- (ProjStruct$ProjectionValues[Points] > 0) | PotentialPoints[Points]
    }
    next()
  }

  # Counting points at the end
  if(i == length(BrNodes)){
    if(Reverse){
      PotentialPoints[Points] <- (ProjStruct$ProjectionValues[Points] > 0) | PotentialPoints[Points]
    } else {
      PotentialPoints[Points] <- (ProjStruct$ProjectionValues[Points] < 1) | PotentialPoints[Points]
    }
    next()
  }

  # all the other cases
  PotentialPoints[Points] <- (ProjStruct$ProjectionValues[Points] > 0 & ProjStruct$ProjectionValues[Points] < 1) | PotentialPoints[Points] 
}

PointsOnEdgesLeaf <- PotentialPoints

if(NodeNames[1] %in% Leaves){
  PointsOnEdgesLeaf <- PointsOnEdgesLeaf | StartOnNode
}

if(NodeNames[length(NodeNames)] %in% Leaves){
  PointsOnEdgesLeaf <- PointsOnEdgesLeaf | EndOnNode
}

return(
  c(
    PointsOnEdges = sum(PotentialPoints),
    PointsOnEdgeExtBoth = sum(PotentialPoints | StartOnNode | EndOnNode),
    PointsOnEdgesLeaf = sum(PointsOnEdgesLeaf),
    EdgesCount = length(BrNodes) - 1,
    EdgesLen = EdgLen
  )
)
})


# Now all the information has been pre-computed and it is possible to filter

if(Mode == "PointNumber"){
ToFilter <- sapply(AllBrInfo, "[[", "PointsOnEdges") < ControlPar
}

if(Mode == "PointNumber_Extrema"){
ToFilter <- sapply(AllBrInfo, "[[", "PointsOnEdgeExtBoth") < ControlPar
}

if(Mode == "PointNumber_Leaves"){
ToFilter <- sapply(AllBrInfo, "[[", "PointsOnEdgesLeaf") < ControlPar
}

if(Mode == "EdgesNumber"){
ToFilter <- sapply(AllBrInfo, "[[", "EdgesCount") < ControlPar
}

if(Mode == "EdgesLength"){
ToFilter <- sapply(AllBrInfo, "[[", "EdgesLen") < ControlPar
}

# Nothing to filter
if(sum(ToFilter)==0){
return(
  list(
    Edges = TargetPG$Edges$Edges,
    Nodes = TargetPG$NodePositions
  )
)

return(list(TargetPG))
}

# TargetPG_New <- TargetPG
# NodesToRemove <- NULL

# Transform Branches in a list of vectors of names
Branches_Names <- lapply(Branches, names)

# Keep track of all the nodes to remove
AllNodes_InternalBranches <- NULL

# For all the branches
for(i in 1:length(ToFilter)){

# If we need to filter this
if(ToFilter[i] == TRUE){

  NodeNames <- Branches_Names[[i]]

  # Is it a final branch ? 
  if(any(NodeNames[c(1, length(NodeNames))] %in% Leaves)){
    # It's a terminal branch, we can safely take it out

    print(paste("Removing the terminal branch with nodes:", paste(NodeNames, collapse = " ")))

    if(length(NodeNames) > 2){
      NodeNames_Ext <- c(
        NodeNames[1],
        rep(NodeNames[-c(1, length(NodeNames))], each = 2),
        NodeNames[length(NodeNames)]
      )
    } else {
      NodeNames_Ext <- NodeNames
    }

    # Set edges to be removed
    for(j in 1:length(NodeNames)){
      igraph::E(Net)$status[igraph::get.edge.ids(graph = Net, vp = NodeNames_Ext)] <- "remove"
    }

  } else {

    # It's a "bridge". We cannot simply remove nodes. Need to introduce a new one by "fusing" two stars

    print(paste("Removing the bridge branch with nodes:", paste(NodeNames, collapse = " ")))

    # Update the list of nodes to update
    AllNodes_InternalBranches <- union(AllNodes_InternalBranches, NodeNames)

  }

  # print(i)
  # print(nrow(TargetPG_New$NodePositions))
  # print(dim(TargetPG_New$ElasticMatrix))

}

}

# Create the network that will contain the final filtered network
Ret_Net <- Net

# Get a net with all the groups of bridges to remove
tNet <- igraph::induced.subgraph(graph = Net, vids = AllNodes_InternalBranches)

if(igraph::vcount(tNet)>0){
# Get the different connected components
CC <- igraph::components(tNet)

# Get the nodes associated with the connected components
Vertex_Comps <- split(names(CC$membership), CC$membership)

# Get the centroid of the different connected components
Centroids <- sapply(Vertex_Comps, function(x){
  colMeans(TargetPG$NodePositions[as.integer(x),])
})

# Prepare a vector that will be used to contract vertices
CVet <- 1:igraph::vcount(Net)

# For each centroid
for(i in 1:length(Vertex_Comps)){
  # Add a new vertex
  Ret_Net <- igraph::add.vertices(
    graph = Ret_Net,
    nv = 1,
    attr = list("name" = paste(igraph::vcount(Ret_Net) + 1))
  )
  # Add a new element to the contraction vector
  CVet <- c(CVet, length(CVet)+1)
  #specify the nodes that will collapse on the new node
  CVet[as.integer(Vertex_Comps[[i]])] <- length(CVet)
}

# collapse the network
Ret_Net <- igraph::contract(graph = Ret_Net, mapping = CVet)
}

# delete edges belonging to the terminal branches
Ret_Net <- igraph::delete.edges(graph = Ret_Net,
                              edges = igraph::E(Ret_Net)[igraph::E(Ret_Net)$status == "remove"])

# remove loops that may have been introduced because of the collapse
Ret_Net <- igraph::simplify(Ret_Net, remove.loops = TRUE)

# Remove empty nodes
Ret_Net <- igraph::induced_subgraph(Ret_Net, igraph::degree(Ret_Net)>0)

# Use the largest index as name of the node
igraph::V(Ret_Net)$name <- lapply(igraph::V(Ret_Net)$name, function(x){
max(as.integer(x))
})

if(igraph::vcount(tNet)>0){
NodeMat <- rbind(TargetPG$NodePositions, t(Centroids))
} else {
NodeMat <- TargetPG$NodePositions
}
rownames(NodeMat) <- NULL

NodeMat <- NodeMat[unlist(igraph::V(Ret_Net)$name), ]

Edges = igraph::get.edgelist(graph = Ret_Net, names = FALSE)
NodesR <- NodeMat
EdgesR <- Edges

[1] "Removing the terminal branch with nodes: 15 18"


In [27]:
i = 0
obj_collapse
epg_collapse_mode
epg_collapse_par
r_elpigraph.CollapseBrances(X = input_data[i], TargetPG = obj_collapse, Mode = epg_collapse_mode, ControlPar = epg_collapse_par)

[1] "Removing the terminal branch with nodes: 3 17"


Edges,[RTYPES.REALSXP]
Nodes,[RTYPES.REALSXP]
